In [1]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
%matplotlib qt


import torchvision.datasets as dsets
import torchvision.transforms as transforms
# from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader
from torch.autograd import Variable

import my_model
from utilities import MyTrainDataSet, MyTestDataSet, load_data_2, load_test_data, min_max_scaling, min_max_unscaling, normalize_one, construct_train_valid_tensor, construct_test_tensor, show_statistic

x mean: 0.5009409709733069 ; std: 0.2436898615684393
y mean: 0.4416590158205868 ; std: 0.23351418998386245
z mean: 0.48521343842977466 ; std: 0.2423616199566707

In [ ]:
# mean and std from training data set
x_mean = 0.5009409709733069
x_std = 0.2436898615684393
y_mean = 0.4416590158205868
y_std = 0.23351418998386245
z_mean = 0.48521343842977466
z_std = 0.2423616199566707

In [2]:
window_size = 6
# ATH = "model/with_early_stop_after_400/model_w_6.pt"
PATH = "model/with_early_stop_after_400_without_normalization/model_w_6.pt"

In [3]:
input_dim = 3
hidden_dim = 100
layer_dim = 1
output_dim = 3

load_model = my_model.LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()
# mmodel = torch.load(PATH)

criterion = nn.MSELoss()

if torch.cuda.is_available():
    load_model.cuda()

In [4]:
test_set_x_1, test_set_y_1, test_set_z_1 = load_test_data('../../../../performance_test/data/test/test_1.csv')
test_set_x_2, test_set_y_2, test_set_z_2 = load_test_data('../../../../performance_test/data/test/test_2.csv')
test_set_x_3, test_set_y_3, test_set_z_3 = load_test_data('../../../../performance_test/data/test/test_3.csv')
test_set_x_4, test_set_y_4, test_set_z_4 = load_test_data('../../../../performance_test/data/test/test_4.csv')
test_set_x_5, test_set_y_5, test_set_z_5 = load_test_data('../../../../performance_test/data/test/test_5.csv')

# do min-max-scaling for each test data set
show_statistic(test_set_x_1)
min_x_1, max_x_1 = min_max_scaling(test_set_x_1)
show_statistic(test_set_x_1)
min_x_2, max_x_2 = min_max_scaling(test_set_x_2)
min_x_3, max_x_3 = min_max_scaling(test_set_x_3)
min_x_4, max_x_4 = min_max_scaling(test_set_x_4)
min_x_5, max_x_5 = min_max_scaling(test_set_x_5)

min_y_1, max_y_1 = min_max_scaling(test_set_y_1)
min_y_2, max_y_2 = min_max_scaling(test_set_y_2)
min_y_3, max_y_3 = min_max_scaling(test_set_y_3)
min_y_4, max_y_4 = min_max_scaling(test_set_y_4)
min_y_5, max_y_5 = min_max_scaling(test_set_y_5)

min_z_1, max_z_1 = min_max_scaling(test_set_z_1)
min_z_2, max_z_2 = min_max_scaling(test_set_z_2)
min_z_3, max_z_3 = min_max_scaling(test_set_z_3)
min_z_4, max_z_4 = min_max_scaling(test_set_z_4)
min_z_5, max_z_5 = min_max_scaling(test_set_z_5)

'''
show_statistic(test_set_x_1)
# do normalization on x of validation test set
normalize_one(test_set_x_1, x_mean, x_std)
show_statistic(test_set_x_1)
normalize_one(test_set_x_2, x_mean, x_std)
normalize_one(test_set_x_3, x_mean, x_std)
normalize_one(test_set_x_4, x_mean, x_std)
normalize_one(test_set_x_5, x_mean, x_std)
# do normalization on y of validation test set
normalize_one(test_set_y_1, y_mean, y_std)
normalize_one(test_set_y_2, y_mean, y_std)
normalize_one(test_set_y_3, y_mean, y_std)
normalize_one(test_set_y_4, y_mean, y_std)
normalize_one(test_set_y_5, y_mean, y_std)
# do normalization on z of validation test set
normalize_one(test_set_z_1, z_mean, z_std)
normalize_one(test_set_z_2, z_mean, z_std)
normalize_one(test_set_z_3, z_mean, z_std)
normalize_one(test_set_z_4, z_mean, z_std)
normalize_one(test_set_z_5, z_mean, z_std)
# show_statistic(train_set_x_1)
'''

test_dataset_1, test_label_1 = construct_test_tensor(test_set_x_1,
                                                     test_set_y_1,
                                                     test_set_z_1,
                                                     window_size)
test_dataset_2, test_label_2 = construct_test_tensor(test_set_x_2,
                                                     test_set_y_2,
                                                     test_set_z_2,
                                                     window_size)
test_dataset_3, test_label_3 = construct_test_tensor(test_set_x_3,
                                                     test_set_y_3,
                                                     test_set_z_3,
                                                     window_size)
test_dataset_4, test_label_4 = construct_test_tensor(test_set_x_4,
                                                     test_set_y_4,
                                                     test_set_z_4,
                                                     window_size)
test_dataset_5, test_label_5 = construct_test_tensor(test_set_x_5,
                                                     test_set_y_5,
                                                     test_set_z_5,
                                                     window_size)

test_set_1 = MyTestDataSet(test_dataset_1, test_label_1)
test_set_2 = MyTestDataSet(test_dataset_2, test_label_2)
test_set_3 = MyTestDataSet(test_dataset_3, test_label_3)
test_set_4 = MyTestDataSet(test_dataset_4, test_label_4)
test_set_5 = MyTestDataSet(test_dataset_5, test_label_5)
print(len(test_set_1))
print(len(test_set_2))
print(len(test_set_3))
print(len(test_set_4))
print(len(test_set_5))
show_statistic(test_set_x_1)
show_statistic(test_set_x_2)
show_statistic(test_set_x_3)
show_statistic(test_set_x_4)
show_statistic(test_set_x_5)


batch_size = 650
test_loader_1 = DataLoader(test_set_1, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
# test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader_2 = DataLoader(test_set_2, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
# test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader_3 = DataLoader(test_set_3, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
# test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader_4 = DataLoader(test_set_4, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
# test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader_5 = DataLoader(test_set_5, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
# test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)

-----------------------------
min: -1.5190000000000001
max: 1.41144
mean: -0.058533702725000004
std: 0.7861266362964188
-----------------------------
min: 0.0
max: 1.0
mean: 0.4983778194656777
std: 0.26826232111779097
394
394
394
394
394
-----------------------------
min: 0.0
max: 1.0
mean: 0.4983778194656777
std: 0.26826232111779097
-----------------------------
min: 0.0
max: 1.0
mean: 0.4999879488895008
std: 0.3247696467306799
-----------------------------
min: 0.0
max: 1.0
mean: 0.49462499118347864
std: 0.2888055402177855
-----------------------------
min: 0.0
max: 1.0
mean: 0.5101133475610831
std: 0.25434808389556507
-----------------------------
min: 0.0
max: 1.0
mean: 0.4851962413697185
std: 0.2642834718504606


In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = np.linspace(0, 15, 1000)
xline = np.sin(zline)
yline = np.cos(zline)
# ax.plot3D(valid_test_set_x_1, valid_test_set_y_1, valid_test_set_z_1, 'gray')
# ax.plot3D(train_set_x_, train_set_y_2, train_set_z_2, 'red')
ax.plot3D(test_set_x_1, test_set_y_1, test_set_z_1, 'red')
# ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = np.linspace(0, 15, 1000)
xline = np.sin(zline)
yline = np.cos(zline)
# ax.plot3D(valid_test_set_x_1, valid_test_set_y_1, valid_test_set_z_1, 'gray')
# ax.plot3D(train_set_x_, train_set_y_2, train_set_z_2, 'red')
ax.plot3D(test_set_x_2, test_set_y_2, test_set_z_2, 'red')
# ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = np.linspace(0, 15, 1000)
xline = np.sin(zline)
yline = np.cos(zline)
# ax.plot3D(valid_test_set_x_1, valid_test_set_y_1, valid_test_set_z_1, 'gray')
# ax.plot3D(train_set_x_, train_set_y_2, train_set_z_2, 'red')
ax.plot3D(test_set_x_3, test_set_y_3, test_set_z_3, 'red')
# ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = np.linspace(0, 15, 1000)
xline = np.sin(zline)
yline = np.cos(zline)
# ax.plot3D(valid_test_set_x_1, valid_test_set_y_1, valid_test_set_z_1, 'gray')
# ax.plot3D(train_set_x_, train_set_y_2, train_set_z_2, 'red')
ax.plot3D(test_set_x_4, test_set_y_4, test_set_z_4, 'red')
# ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = np.linspace(0, 15, 1000)
xline = np.sin(zline)
yline = np.cos(zline)
# ax.plot3D(valid_test_set_x_1, valid_test_set_y_1, valid_test_set_z_1, 'gray')
# ax.plot3D(train_set_x_, train_set_y_2, train_set_z_2, 'red')
ax.plot3D(test_set_x_5, test_set_y_5, test_set_z_5, 'red')
# ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')

In [ ]:
0.017379868775606155

In [5]:
# get test results
seq_dim = window_size
input_dim = 3
# test_seq = []
test_predd = []
# test_gt = []
total_test_loss = 0.0
test_batch = 0
for i, (seqs, labels) in enumerate(test_loader_1):
    if torch.cuda.is_available():
        seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
        labels = Variable(labels.cuda())
    else:
        seqs = Variable(seqs.view(-1, seq_dim, input_dim))
        
    outputs = load_model(seqs)
    loss = criterion(outputs, labels)
    total_test_loss += loss.data.item()
    test_predd.append(outputs)
    test_batch = i + 1

print(total_test_loss/test_batch)

for i in range(len(test_predd)):
    if (i == 0):
        pred = test_predd[i].cpu().detach().numpy()
    else:
        pred = np.append(pred, test_predd[i].cpu().detach().numpy(), axis = 0)
'''        
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(test_set_x_1, test_set_y_1, test_set_z_1, 'gray')
ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')
'''
# scale back to original dimension
min_max_unscaling(test_set_x_1, min_x_1, max_x_1)
min_max_unscaling(test_set_y_1, min_y_1, max_y_1)
min_max_unscaling(test_set_z_1, min_z_1, max_z_1)
pred_set_x_1 = list(pred[:,0])
pred_set_y_1 = list(pred[:,1])
pred_set_z_1 = list(pred[:,2])
min_max_unscaling(pred_set_x_1, min_x_1, max_x_1)
min_max_unscaling(pred_set_y_1, min_y_1, max_y_1)
min_max_unscaling(pred_set_z_1, min_z_1, max_z_1)
# plot to verify
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(test_set_x_1, test_set_y_1, test_set_z_1, 'gray')
ax.plot3D(pred_set_x_1, pred_set_y_1, pred_set_z_1, 'red')
# save to file
file = open('js_predict/predict_test1.txt', 'w')
for i in range(len(pred_set_x_1)):
    if (i != len(pred_set_x_1) - 1):
        file.write(str(pred_set_x_1[i]) + " ")
    else:
        file.write(str(pred_set_x_1[i]) + "\n")
for i in range(len(pred_set_y_1)):
    if (i != len(pred_set_y_1) - 1):
        file.write(str(pred_set_y_1[i]) + " ")
    else:
        file.write(str(pred_set_y_1[i]) + "\n")
for i in range(len(pred_set_z_1)):
    if (i != len(pred_set_z_1) - 1):
        file.write(str(pred_set_z_1[i]) + " ")
    else:
        file.write(str(pred_set_z_1[i]) + "\n")
file.close()

0.0010135570773854852


In [ ]:
0.0010135570773854852

In [6]:
# get test results
seq_dim = window_size
input_dim = 3
# test_seq = []
test_predd = []
# test_gt = []
total_test_loss = 0.0
test_batch = 0
for i, (seqs, labels) in enumerate(test_loader_2):
    if torch.cuda.is_available():
        seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
        labels = Variable(labels.cuda())
    else:
        seqs = Variable(seqs.view(-1, seq_dim, input_dim))
        
    outputs = load_model(seqs)
    loss = criterion(outputs, labels)
    total_test_loss += loss.data.item()
    test_predd.append(outputs)
    test_batch = i + 1

print(total_test_loss/test_batch)

for i in range(len(test_predd)):
    if (i == 0):
        pred = test_predd[i].cpu().detach().numpy()
    else:
        pred = np.append(pred, test_predd[i].cpu().detach().numpy(), axis = 0)
'''
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(test_set_x_2, test_set_y_2, test_set_z_2, 'gray')
ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')
'''
# scale back to original dimension
min_max_unscaling(test_set_x_2, min_x_2, max_x_2)
min_max_unscaling(test_set_y_2, min_y_2, max_y_2)
min_max_unscaling(test_set_z_2, min_z_2, max_z_2)
pred_set_x_2 = list(pred[:,0])
pred_set_y_2 = list(pred[:,1])
pred_set_z_2 = list(pred[:,2])
min_max_unscaling(pred_set_x_2, min_x_2, max_x_2)
min_max_unscaling(pred_set_y_2, min_y_2, max_y_2)
min_max_unscaling(pred_set_z_2, min_z_2, max_z_2)
# plot to verify
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(test_set_x_2, test_set_y_2, test_set_z_2, 'gray')
ax.plot3D(pred_set_x_2, pred_set_y_2, pred_set_z_2, 'red')
# save to file
file = open('js_predict/predict_test2.txt', 'w')
for i in range(len(pred_set_x_2)):
    if (i != len(pred_set_x_2) - 1):
        file.write(str(pred_set_x_2[i]) + " ")
    else:
        file.write(str(pred_set_x_2[i]) + "\n")
for i in range(len(pred_set_y_2)):
    if (i != len(pred_set_y_2) - 1):
        file.write(str(pred_set_y_2[i]) + " ")
    else:
        file.write(str(pred_set_y_2[i]) + "\n")
for i in range(len(pred_set_z_2)):
    if (i != len(pred_set_z_2) - 1):
        file.write(str(pred_set_z_2[i]) + " ")
    else:
        file.write(str(pred_set_z_2[i]) + "\n")
file.close()

0.0009599363547749817


In [ ]:
0.0009599363547749817

In [7]:
# get test results
seq_dim = window_size
input_dim = 3
# test_seq = []
test_predd = []
# test_gt = []
total_test_loss = 0.0
test_batch = 0
for i, (seqs, labels) in enumerate(test_loader_3):
    if torch.cuda.is_available():
        seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
        labels = Variable(labels.cuda())
    else:
        seqs = Variable(seqs.view(-1, seq_dim, input_dim))
        
    outputs = load_model(seqs)
    loss = criterion(outputs, labels)
    total_test_loss += loss.data.item()
    test_predd.append(outputs)
    test_batch = i + 1

print(total_test_loss/test_batch)

for i in range(len(test_predd)):
    if (i == 0):
        pred = test_predd[i].cpu().detach().numpy()
    else:
        pred = np.append(pred, test_predd[i].cpu().detach().numpy(), axis = 0)
'''
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(test_set_x_3, test_set_y_3, test_set_z_3, 'gray')
ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')
'''
# scale back to original dimension
min_max_unscaling(test_set_x_3, min_x_3, max_x_3)
min_max_unscaling(test_set_y_3, min_y_3, max_y_3)
min_max_unscaling(test_set_z_3, min_z_3, max_z_3)
pred_set_x_3 = list(pred[:,0])
pred_set_y_3 = list(pred[:,1])
pred_set_z_3 = list(pred[:,2])
min_max_unscaling(pred_set_x_3, min_x_3, max_x_3)
min_max_unscaling(pred_set_y_3, min_y_3, max_y_3)
min_max_unscaling(pred_set_z_3, min_z_3, max_z_3)
# plot to verify
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(test_set_x_3, test_set_y_3, test_set_z_3, 'gray')
ax.plot3D(pred_set_x_3, pred_set_y_3, pred_set_z_3, 'red')
# save to file
file = open('js_predict/predict_test3.txt', 'w')
for i in range(len(pred_set_x_3)):
    if (i != len(pred_set_x_3) - 1):
        file.write(str(pred_set_x_3[i]) + " ")
    else:
        file.write(str(pred_set_x_3[i]) + "\n")
for i in range(len(pred_set_y_3)):
    if (i != len(pred_set_y_3) - 1):
        file.write(str(pred_set_y_3[i]) + " ")
    else:
        file.write(str(pred_set_y_3[i]) + "\n")
for i in range(len(pred_set_z_3)):
    if (i != len(pred_set_z_3) - 1):
        file.write(str(pred_set_z_3[i]) + " ")
    else:
        file.write(str(pred_set_z_3[i]) + "\n")
file.close()

0.0020637575071305037


In [ ]:
0.0020637575071305037

In [8]:
# get test results
seq_dim = window_size
input_dim = 3
# test_seq = []
test_predd = []
# test_gt = []
total_test_loss = 0.0
test_batch = 0
for i, (seqs, labels) in enumerate(test_loader_4):
    if torch.cuda.is_available():
        seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
        labels = Variable(labels.cuda())
    else:
        seqs = Variable(seqs.view(-1, seq_dim, input_dim))
        
    outputs = load_model(seqs)
    loss = criterion(outputs, labels)
    total_test_loss += loss.data.item()
    test_predd.append(outputs)
    test_batch = i + 1

print(total_test_loss/test_batch)

for i in range(len(test_predd)):
    if (i == 0):
        pred = test_predd[i].cpu().detach().numpy()
    else:
        pred = np.append(pred, test_predd[i].cpu().detach().numpy(), axis = 0)
'''
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(test_set_x_4, test_set_y_4, test_set_z_4, 'gray')
ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')
'''
# scale back to original dimension
min_max_unscaling(test_set_x_4, min_x_4, max_x_4)
min_max_unscaling(test_set_y_4, min_y_4, max_y_4)
min_max_unscaling(test_set_z_4, min_z_4, max_z_4)
pred_set_x_4 = list(pred[:,0])
pred_set_y_4 = list(pred[:,1])
pred_set_z_4 = list(pred[:,2])
min_max_unscaling(pred_set_x_4, min_x_4, max_x_4)
min_max_unscaling(pred_set_y_4, min_y_4, max_y_4)
min_max_unscaling(pred_set_z_4, min_z_4, max_z_4)
# plot to verify
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(test_set_x_4, test_set_y_4, test_set_z_4, 'gray')
ax.plot3D(pred_set_x_4, pred_set_y_4, pred_set_z_4, 'red')
# save to file
file = open('js_predict/predict_test4.txt', 'w')
for i in range(len(pred_set_x_4)):
    if (i != len(pred_set_x_4) - 1):
        file.write(str(pred_set_x_4[i]) + " ")
    else:
        file.write(str(pred_set_x_4[i]) + "\n")
for i in range(len(pred_set_y_4)):
    if (i != len(pred_set_y_4) - 1):
        file.write(str(pred_set_y_4[i]) + " ")
    else:
        file.write(str(pred_set_y_4[i]) + "\n")
for i in range(len(pred_set_z_4)):
    if (i != len(pred_set_z_4) - 1):
        file.write(str(pred_set_z_4[i]) + " ")
    else:
        file.write(str(pred_set_z_4[i]) + "\n")
file.close()

0.003436723491176963


In [ ]:
0.003436723491176963

In [9]:
# get test results
seq_dim = window_size
input_dim = 3
# test_seq = []
test_predd = []
# test_gt = []
total_test_loss = 0.0
test_batch = 0
for i, (seqs, labels) in enumerate(test_loader_5):
    if torch.cuda.is_available():
        seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
        labels = Variable(labels.cuda())
    else:
        seqs = Variable(seqs.view(-1, seq_dim, input_dim))
        
    outputs = load_model(seqs)
    loss = criterion(outputs, labels)
    total_test_loss += loss.data.item()
    test_predd.append(outputs)
    test_batch = i + 1

print(total_test_loss/test_batch)

for i in range(len(test_predd)):
    if (i == 0):
        pred = test_predd[i].cpu().detach().numpy()
    else:
        pred = np.append(pred, test_predd[i].cpu().detach().numpy(), axis = 0)
'''
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(test_set_x_5, test_set_y_5, test_set_z_5, 'gray')
ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')
'''
# scale back to original dimension
min_max_unscaling(test_set_x_5, min_x_5, max_x_5)
min_max_unscaling(test_set_y_5, min_y_5, max_y_5)
min_max_unscaling(test_set_z_5, min_z_5, max_z_5)
pred_set_x_5 = list(pred[:,0])
pred_set_y_5 = list(pred[:,1])
pred_set_z_5 = list(pred[:,2])
min_max_unscaling(pred_set_x_5, min_x_5, max_x_5)
min_max_unscaling(pred_set_y_5, min_y_5, max_y_5)
min_max_unscaling(pred_set_z_5, min_z_5, max_z_5)
# plot to verify
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(test_set_x_5, test_set_y_5, test_set_z_5, 'gray')
ax.plot3D(pred_set_x_5, pred_set_y_5, pred_set_z_5, 'red')
# save to file
file = open('js_predict/predict_test5.txt', 'w')
for i in range(len(pred_set_x_5)):
    if (i != len(pred_set_x_5) - 1):
        file.write(str(pred_set_x_5[i]) + " ")
    else:
        file.write(str(pred_set_x_5[i]) + "\n")
for i in range(len(pred_set_y_5)):
    if (i != len(pred_set_y_5) - 1):
        file.write(str(pred_set_y_5[i]) + " ")
    else:
        file.write(str(pred_set_y_5[i]) + "\n")
for i in range(len(pred_set_z_5)):
    if (i != len(pred_set_z_5) - 1):
        file.write(str(pred_set_z_5[i]) + " ")
    else:
        file.write(str(pred_set_z_5[i]) + "\n")
file.close()

0.006062280386686325


In [ ]:
0.006062280386686325